## This notebook, I will check the MAPE on Linear Regression and the use prediction as same value from last value, then check the MAPE:

I have already done in TF Dense layer so I will add those rsult on the data frame I already have. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from helper_functions import *

In [2]:
data = pd.read_csv("result_dense_with_category.csv")
train = pd.read_csv("train.csv")

In [3]:
data.head()

,Country,MAPE,Density,Active,forecast,category
0,1001,1.504614,3.334431,1417,[3.3091352 3.31727028 3.32801533 3.33693123 3...,2
1,1003,4.143341,7.823300,13401,[7.85833788 7.88518858 7.90744925 7.90828514 7...,2
2,1005,1.395184,1.206827,239,[1.16141069 1.16513038 1.17019248 1.18426347 1...,1
3,1007,5.228748,1.236650,220,[1.22299695 1.22010887 1.22093809 1.21361172 1...,2
4,1009,1.442244,1.777708,789,[1.75237012 1.76360607 1.77065146 1.7765491 1...,1


In [4]:
# Get Linear Regression result (MAPE)
IDS = train.cfips.unique()
x_train = np.arange(31).reshape((-1,1))
x_test = np.arange(31,39).reshape((-1,1))

# Based on the Chris's rule, SMAPE > THRESHOLD -> Use linear regrssion 

mape = []
predict = []


for cfips in IDS:
    c = cfips
    df = train.loc[train['cfips'] == c]
    last = df['microbusiness_density'].values[-9]

    # Fit linear regression
    model = LinearRegression()
    model.fit(x_train,df['microbusiness_density'][:31])
    p = model.predict(x_train)

    # Compute train error
    err = p - df['microbusiness_density'][:31].values
    rng = df['microbusiness_density'].max() - df['microbusiness_density'].min()
    #print(f'Error: {err}\nRange: {rng}')

    # Determin if time series is linear or not
    s = 0
    for k in range(31):
        e = np.abs(err[k])
        #print(f'e : {e}')
        r = e / rng # absolute error divided by range
        #print(f'r: {r}')
        s += r
    s = s/39 # now S is MAPE mean absolute percentage error
    smape.append(s) 
   

    # Infer test data with linear regression
    p2 = model.predict(x_test)
    shift = last - p2[0]
    # Get Renear Regression results    
    preds = p2[:]+shift
    label = 0
    lr_or_non_lr.append(label)
    test_label = np.expand_dims(np.array(train[train['cfips'] == c]['microbusiness_density'][31:39]), axis=0)
    results = evaluate_preds(test_label, preds)
    m = results['mape']
    predict.append(np.array(preds).reshape((-1)))
    mape.append(m)
    new_predict.append(preds)
        


data['LR_MAPE'] = mape

The data predict with Linear Regression: 3135
The data not predict with Linear Regression: 0
Model: LinearRegression()
Mean SMAPE: 0.11540516723841959
Min SMAPE: 0.0018782232910424662
Max SMAPE: 0.2993328663980672


In [5]:
# Result LR_MAPE is only linear regression model result. 
data.head()

,Country,MAPE,Density,Active,forecast,category,LR_MAPE
0,1001,1.504614,3.334431,1417,[3.3091352 3.31727028 3.32801533 3.33693123 3...,2,0.992271
1,1003,4.143341,7.823300,13401,[7.85833788 7.88518858 7.90744925 7.90828514 7...,2,4.317589
2,1005,1.395184,1.206827,239,[1.16141069 1.16513038 1.17019248 1.18426347 1...,1,1.833943
3,1007,5.228748,1.236650,220,[1.22299695 1.22010887 1.22093809 1.21361172 1...,2,4.259990
4,1009,1.442244,1.777708,789,[1.75237012 1.76360607 1.77065146 1.7765491 1...,1,1.285277


In [6]:
# Next get rsult with all the same values
IDS = train.cfips.unique()
x_train = np.arange(31).reshape((-1,1))
x_test = np.arange(31,39).reshape((-1,1))

# Based on the Chris's rule, SMAPE > THRESHOLD -> Use linear regrssion 
lr_or_non_lr = []  # If use linear regression, label as 1, if not label as 0
smape = []
mape = []
predict = []
new_predict = []

for cfips in IDS:
    c = cfips
    
    df = train.loc[train['cfips'] == c]
    last = df['microbusiness_density'].values[-9]


    model = 'last_value'
    preds = [last]*8
    label = 1
    lr_or_non_lr.append(label)
    test_label = np.expand_dims(np.array(train[train['cfips'] == c]['microbusiness_density'][31:39]), axis=0)
    results = evaluate_preds(test_label, preds)
    m = results['mape']
    predict.append(np.array(preds).reshape((-1)))
    mape.append(m)
    new_predict.append(preds)

count_lr = lr_or_non_lr.count(0)
count_non_lr = lr_or_non_lr.count(1)
print(f'The data predict with Linear Regression: {count_lr}')
print(f'The data not predict with Linear Regression: {count_non_lr}')

data['SV_MAPE'] = mape

The data predict with Linear Regression: 0
The data not predict with Linear Regression: 3135


In [7]:
mape_data = data[['Country', 'MAPE', 'LR_MAPE', 'SV_MAPE']]
mape_data.head()

,Country,MAPE,LR_MAPE,SV_MAPE
0,1001,1.504614,0.992271,1.835232
1,1003,4.143341,4.317589,5.230218
2,1005,1.395184,1.833943,1.101416
3,1007,5.228748,4.259990,3.574082
4,1009,1.442244,1.285277,2.116541


In [8]:
# Compare those three numbers and categorize by the best score. 
best_scores = mape_data.min(axis='columns')

In [9]:
cat = []
for i, m in enumerate(best_scores):
    if m == data['MAPE'].iloc[i]:
        category = 'tf_dense'
        cat.append(category)
    elif m == data["LR_MAPE"].iloc[i]:
        category = "LR"
        cat.append(category)
    elif m == data['SV_MAPE'].iloc[i]:
        category = "SV"
        cat.append(category)
data['category'] = cat
data.head()

,Country,MAPE,Density,Active,forecast,category,LR_MAPE,SV_MAPE
0,1001,1.504614,3.334431,1417,[3.3091352 3.31727028 3.32801533 3.33693123 3...,LR,0.992271,1.835232
1,1003,4.143341,7.823300,13401,[7.85833788 7.88518858 7.90744925 7.90828514 7...,tf_dense,4.317589,5.230218
2,1005,1.395184,1.206827,239,[1.16141069 1.16513038 1.17019248 1.18426347 1...,SV,1.833943,1.101416
3,1007,5.228748,1.236650,220,[1.22299695 1.22010887 1.22093809 1.21361172 1...,SV,4.259990,3.574082
4,1009,1.442244,1.777708,789,[1.75237012 1.76360607 1.77065146 1.7765491 1...,LR,1.285277,2.116541


In [10]:
data.category.value_counts()

SV          1486
LR           851
tf_dense     798
Name: category, dtype: int64

In [11]:
data.to_csv('compare_mape.csv', index=False)

## As you see the results, I got the best score for 3 different ways of forecasting. Next step is to train with based on the result and get the forecasts. 